In [1]:
import os
import sys
this = os.getcwd()
path = this[:this.rfind("/")]
if not path in sys.path: sys.path.append(path)
xgboost_path = path + "/xgboost_regressor"
promedios_path = path + "/promedio_zona"

import pandas as pd
import matplotlib
from datos import FEATURES_DISPONIBLES
from modelo import Modelo

pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
from xgboost_regressor.xgboost_predictor import XGBoostRegressor
from promedio_zona.promedio_zona import PromedioZona

In [3]:
class EnsamblePromedioXGBoostConcatenados(XGBoostRegressor):
    """
        Usa el resultado de las predicciones del modelo PromedioZona
        para entrenar y predecir con un XGBoostRegressor.
    """
    
    
    @Modelo.cronometrar()
    def __init__(self):
        self.modelo_promedios = PromedioZona()
        super().__init__()        
        
    @Modelo.cronometrar()
    def cargar_datos(self):
        self.modelo_promedios.cargar_datos()
        super().cargar_datos()
    
    @Modelo.cronometrar()
    def entrenar(self):
        self.modelo_promedios.entrenar()
        predicciones_train =  self.modelo_promedios.predecir(self.modelo_promedios.train_data)
        predicciones_test = self.modelo_promedios.predecir(self.modelo_promedios.test_data)
        predicciones_submit = self.modelo_promedios.predecir(self.modelo_promedios.submit_data)
        self.train_data['prediccion_promedios'] = predicciones_train['target']
        self.test_data['prediccion_promedios'] = predicciones_test['target']
        self.submit_data['prediccion_promedios'] = predicciones_submit['target']
        super().entrenar()


In [5]:
ensamble = EnsamblePromedioXGBoostConcatenados()

instanciar demoro 0.0 segundos
instanciar demoro 0.0 segundos
__init__ demoro 0.0 segundos


In [6]:
ensamble.cargar_datos()

cargar demoro 1.35 segundos
cargar demoro 62.45 segundos
cargar_datos demoro 67.64 segundos


In [7]:
ensamble.entrenar()

entrenar demoro 0.0 segundos
entrenar demoro 0.15 segundos
predecir demoro 20.71 segundos
predecir demoro 7.16 segundos
predecir demoro 7.21 segundos
[21:53:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
entrenar demoro 0.0 segundos
entrenar demoro 286.01 segundos
entrenar demoro 321.25 segundos


In [8]:
ensamble.validar()

predecir demoro 3.02 segundos
puntuar demoro 0.0 segundos
validar demoro 3.03 segundos


624058.0

In [9]:
predicciones = ensamble.predecir(ensamble.submit_data)

predecir demoro 3.09 segundos


In [10]:
comentario = "ensamble promedios + xgboost (concatenados) - con analisis de titulo y descripcion - score local 624058"
ensamble.presentar(predicciones, comentario)

guardar demoro 0.13 segundos


100%|██████████| 968k/968k [00:04<00:00, 203kB/s]  


submit demoro 7.01 segundos
presentar demoro 7.14 segundos
